# The Roboat Environmental Data API Wrapper

The following is a very, very small intro into the Roboat environmental API wrapper. It should give you a good idea of what is possible and will be completely documented in due time.

In the near future, the wrapper will be updated with object functionality to make everything super fluid - in the mean time, we can get by with this simple API wrapper with somewhat limited functionality and little documentation...


## Authentication

To get started, you must first request an API Key. You can do so at https://www.roboat-enviro.com/api-keys. Once you have your API key, store it as an environment variable called *ROBOAT-ENVIRO_APIKEY*.

In [16]:
import json
from pprint import pprint
import pandas as pd
from datetime import datetime, timedelta

In [17]:
import roboatenviro
print ("roboatenviro v{}".format(roboatenviro.__version__))

roboatenviro v0.1.0


## Advanced Queries

The API itself has quite powerful querying capabilites, though they can be a bit confusing to use at first. Eventually, I will get around to writing a "helper" function to make this as simple as possible.

### Limit

To limit the number of results you get for queries where a list of information is returned, you can add the `limit` keyword to your `params` dictionary. The argument must be an integer (i.e. `limit=5`.)

Example:

Return the first five sensors in the list:

    >>> api.get_sensors(params=dict(limit=5))
    
### Sort

You can sort the order of results by any column and either by `asc` or `desc`. The format for the `sort` keyword must be `sort="[column],[asc or desc]"`. You can also join multiple sorts together using a semicolon.

Example:

Return the first five sensors, sorted asc by the last seen column:

    >>> api.get_sensors(params=dict(limit=5, sort="last_seen,asc"))
    
  
Return the first five sensors, sorted asc by last_seen and asc by username:

    >>> api.get_sensors(params=dict(limit=5, sort="last_seen,asc;username,asc"))
    

### Filter

Filter is probably the most useful because it allows you to grab data between certain timestamps! There is also a ton of embedded functionality built in. The arguments that can be used include:

  * `eq`: equals
  * `ne`: not equals
  * `lt`: less than
  * `le`: less than or equal to
  * `gt`: greater than
  * `ge`: greater than or equal to
  * `in`: in
  * `like`: like
  
The format to the argument is as follows: `filter="[column],[arg],[value]"`. 

Examples:

Grab data between May 1st and June 1st, 2018.

    >>> api.get_trf_data(sn="SN001", params=dict(filter="timestamp,ge,2018-05-01;timestamp,lt,2018-06-01"))


## Initialize a RoboatEnviro object

To start, we will first setup an object of the manager, called `roboatenviro.RoboatEnviro`. If you haven't stored your api key as an environment variable, or have called it something else, feel free to enter it here.

In [18]:
import os
# get the token from a different environment variable
#token = os.getenv("ROBOAT-ENVIRO_APIKEY_DEV")

#hardcoded in here for now...
token = "token"

# set up the roboatenviro.RoboatEnviro object
# here, we are also going to manually set the endpoint because I am doing this tutorial on 
# localhost...
api = roboatenviro.RoboatEnviro(token=token, endpoint="http://localhost:5000/api/")

## `roboatenviro.RoboatEnviro(*args, **kwargs)`

The RoboatEnviro (can also be called the 'manager') object provides a simple way to view information about all the sensors you have access to with your account. Remember, the API key is tied to your account, so don't share the key with anyone!  Please keep it safe and sound.

Below, I will detail a few of the methods available from the manager:

### `roboatenviro.legacy.RoboatEnviro.get_users(return_type='json')`

The `get_account` method simply retrieves the account information tied to your API key as a dictionary.

In [22]:
api.get_users(return_type='dataframe')
#api.add_user() #TODO
#api.update_user() #TODO
#api.delete_user("drew")

http://localhost:5000/api/v1/users token GET
<Response [200]>


""


### `roboatenviro.RoboatEnviro.add_sensor(**kwargs)`

Use this method to add a sensor to the db.

In [25]:
api.get_sensors()
api.add_sensor(params=dict(sn="SN001"))

http://localhost:5000/api/v1/sensors token GET
<Response [200]>
http://localhost:5000/api/v1/sensors/ token POST


<Response [500]>

### `roboatenviro.RoboatEnviro.get_sensors(return_type='json', **kwargs)`

With the `get_sensors` method, we can retrieve all the sensors we have access to. You can return them either as 'json' or 'dataframe'.

#### json

In [26]:
api.get_sensors()

http://localhost:5000/api/v1/sensors token GET
<Response [200]>


[{'created_at': 'Thu, 09 Jan 2020 01:42:49 GMT', 'id': 8, 'sn': 'SN001'}]

#### dataframe

In [27]:
api.get_sensors(return_type='dataframe')

http://localhost:5000/api/v1/sensors token GET
<Response [200]>


,created_at,id,sn
0,"Thu, 09 Jan 2020 01:42:49 GMT",8,SN001


You should also be able to send whatever query params you want as keywords - see above in the "Advanced Queries" section for more information.

In [28]:
api.get_sensors(return_type='dataframe', params=dict(limit=1))

http://localhost:5000/api/v1/sensors token GET
<Response [200]>


,created_at,id,sn
0,"Thu, 09 Jan 2020 01:42:49 GMT",8,SN001


### `roboatenviro.RoboatEnviro.get_sensor(sn)`

You can get a single sensor, by using this method.

In [29]:
api.get_sensor(sn="SN001")

http://localhost:5000/api/v1/sensors/SN001 token GET
<Response [200]>


{'created_at': '2020-01-09T01:42:49.478422',
 'id': 8,
 'modified_at': '2020-01-09T01:42:49.478429',
 'sn': 'SN001'}

### `roboatenviro.RoboatEnviro.update_sensor(sn, params)`

Use this method to update a sensor with params as a dictionary.

In [30]:
#params = dict(lat=12.1, lon=-100)
params = dict(sn="SN002")
api.update_sensor("SN001", params=params)
params = dict(sn="SN001")
api.update_sensor("SN002", params=params)

http://localhost:5000/api/v1/sensors/SN001 token PUT
<Response [200]>
http://localhost:5000/api/v1/sensors/SN002 token PUT
<Response [200]>


{'created_at': '2020-01-09T01:42:49.478422',
 'id': 8,
 'modified_at': '2020-01-09T01:43:12.504173',
 'sn': 'SN001'}

### `roboatenviro.RoboatEnviro.delete_sensor(sn, params)`

Use this method to delete a sensor (CAUTION: deletion cascades to all related tables).

In [31]:
#api.delete_sensor(sn="SN001")

### `roboatenviro.RoboatEnviro.add_trf_scan_set(**kwargs)`

Use this method to add a TRF scan set to the db.

In [32]:
df = pd.read_csv("trf_upload.csv")
df["scan_datetime"] = datetime.utcnow().isoformat()
df.drop(["sensor_sn"], axis=1, inplace=True)
params = {
    "sensor_sn": "SN001",
    "start_wl": int(df["mcr_wl"].min()),
    "end_wl": int(df["mcr_wl"].max()),
    "start_datetime": df["scan_datetime"].min(),
    "end_datetime": df["scan_datetime"].max()
}
scan_set = api.add_trf_scan_set(params=params)
scan_set = json.loads(scan_set.text)
scan_set_id = scan_set['id']
print("scan_set_id: ", scan_set_id)
df['scan_set_id'] = scan_set_id
display(df.head())

http://localhost:5000/api/v1/trf-scan-sets/ token POST
scan_set_id:  37


,mcr_wl,trf_scan,scan_datetime,scan_set_id
0,325,"[0.04500000000000052, 0.04500000000000052, 0.0...",2020-01-09T01:43:20.115608,37
1,326,"[0.04500000000000052, 0.04500000000000052, 0.0...",2020-01-09T01:43:20.115608,37
2,327,"[0.04500000000000052, 0.04500000000000052, 0.0...",2020-01-09T01:43:20.115608,37
3,328,"[0.04500000000000052, 0.04500000000000052, 0.0...",2020-01-09T01:43:20.115608,37
4,329,"[0.04500000000000052, 0.04500000000000052, 0.0...",2020-01-09T01:43:20.115608,37


### `roboatenviro.RoboatEnviro.add_trf_data(**kwargs)`

Use this method to add a TRF scans to the db.

In [33]:
for idx, row in df.iterrows():
    params = row.to_dict()
    api.add_trf_data(params=params)

http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1/raw-trf-data/ token POST
http://localhost:5000/api/v1

### `roboatenviro.RoboatEnviro.get_trf_scan_set(sn, return_type="json", final_data=True, **kwargs)`

Use this method to retrieve a list of TRF scan sets for a given SN.

In [37]:
display(api.get_trf_scan_set("SN001", return_type="dataframe"))

http://localhost:5000/api/v1/trf-scan-sets/ token GET
<Response [200]>


,end_datetime,end_wl,id,sensor_sn,start_datetime,start_wl
0,"Thu, 09 Jan 2020 01:43:20 GMT",350,37,SN001,"Thu, 09 Jan 2020 01:43:20 GMT",325


### `roboatenviro.RoboatEnviro.get_trf_data(sn, return_type="json", final_data=True, **kwargs)`

Use this method to retrieve a list of TRF data for a given SN.

In [35]:
display(api.get_trf_data(sn="SN001", return_type="dataframe"))

http://localhost:5000/api/v1/raw-trf-data/ token GET
<Response [200]>


,id,mcr_wl,scan_datetime,scan_set_id,trf_scan
0,268,325,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
1,269,326,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
2,270,327,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
3,271,328,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
4,272,329,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
5,273,330,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
6,274,331,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
7,275,332,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
8,276,333,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
9,277,334,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."


Get the last 15 datapoints since January 1st:

In [38]:
display(api.get_trf_data(sn="SN001", return_type="dataframe",
                 params=dict(filter="scan_datetime,gt,2020-01-08 17:46:19", limit=15)))

http://localhost:5000/api/v1/raw-trf-data/ token GET
<Response [200]>


,id,mcr_wl,scan_datetime,scan_set_id,trf_scan
0,268,325,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
1,269,326,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
2,270,327,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
3,271,328,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
4,272,329,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
5,273,330,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
6,274,331,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
7,275,332,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
8,276,333,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."
9,277,334,"Thu, 09 Jan 2020 01:43:20 GMT",37,"[0.04500000000000052, 0.04500000000000052, 0.0..."


In [ ]:
#api.delete_trf_scan_sets() #TODO

### `roboatenviro.RoboatEnviro.get_ssf_data(sn, return_type="json", final_data=True, **kwargs)`

Use this method to retrieve a list of SSF data for a given SN.

In [40]:
#api.add_ssf_data()
display(api.get_ssf_data(sn="SN001", return_type='json'))
#api.delete_ssf_data()

http://localhost:5000/api/v1/raw-ssf-data/ token GET
<Response [200]>


[]

### `roboatenviro.RoboatEnviro.get_sensor_logs(return_type="json", **kwargs)`

Use this method to retrieve a list of sensor logs.

In [ ]:
params = {
    "sensor_sn": "SN001",
    "log_datetime": datetime.utcnow().isoformat(),
    "log": "test log",
    "priority": "LOW"
}
api.add_sensor_log(params=params)
api.get_sensor_logs(return_type='dataframe')
#api.delete_sensor_log() #TODO

In [ ]:
params = {
    "sensor_sn": "SN001",
    "metadata_datetime": datetime.utcnow().isoformat(),
    "sensor_metadata": "test metadata",
}
api.add_sensor_metadata(params=params)
api.get_sensor_metadata(return_type='dataframe')
#api.delete_sensor_metadata() #TODO

### `roboatenviro.RoboatEnviro.get_deployments(return_type="json", **kwargs)`

Use this method to retrieve a list of deployments.

In [15]:
params = {
    "name": "Test deployment",
    "start_datetime": datetime.utcnow().isoformat(),
    "end_datetime": (datetime.utcnow() + timedelta(days=1)).isoformat(),
    "summary": "Just a test"
}
#api.add_deployment(params=params)
#api.add_deployment_sensors(param=params)
api.get_deployments(return_type='dataframe')
#api.get_deployment_sensors(return_type='dataframe')
#api.delete_deployment()

http://localhost:5000/api/v1/deployments token GET
<Response [200]>


,end_datetime,id,name,start_datetime,summary
0,"Thu, 09 Jan 2020 20:55:55 GMT",1,Test deployment,"Wed, 08 Jan 2020 20:55:55 GMT",Just a test


In [ ]:
#visualize trf scan sets, animations of timeseries + stacked line plot
#visulize ssf scans